In [10]:
initial_path = 'peptide-QML/'
# initial_path = '../'

In [11]:
import numpy as np
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import functions as f
from my_code import pytorch_model as ptm
from my_code import quantum_nodes as qn
from my_code import pytorch_wrappers as pw

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
dim = 12
day = "0905"


# -- DATA -- #

# name of the notebook
name_notebook = "{}-{}c_h{}.ipynb".format(day, dim)

# load data
data_file_path = '{}data/Scores/PET/PET_SCORES'.format(initial_path)
string_list, number_list = f.read_data_file(
    data_file_path,
    skip_first_line=True, 
    delimiter=',', 
    columns_to_return=[1, 2],
    which_columns_are_numbers=[2]
    )
score_list = np.array(number_list)/100
vector_list = np.array([f.string_to_vector(string) for string in string_list]) 
X, Y, X_validation, Y_validation = f.create_validating_set(vector_list, score_list, percentage=0.1)

# Define the dataset
input_data = ptm.torch.tensor(X) 
target_data = ptm.torch.tensor(Y).view(-1, 1)

# Define the validation set
input_validation = ptm.torch.tensor(X_validation)
target_validation = ptm.torch.tensor(Y_validation).view(-1, 1)


# -- TRAINING LOOP -- #
for iii in range(5):

    for block_n_layers in range(1, 21):
        print(iii, " # ------- block_n_layers: ", block_n_layers, " ------- #")
        print("")
        print("")

        # -- CLASSICAL MODEL -- #
        n_aminoacids = len(string_list[0])
        input_dim = input_data.shape[1]

        layers_dim = np.linspace(input_dim, 1, block_n_layers+1).astype(int)
        layers = []
        for i, o in zip(layers_dim[:-1], layers_dim[1:]):
            print(i, o)
            layers += [ptm.nn.Linear(i, o), ptm.nn.Sigmoid()]    # activation function RELU didn't work well  
        layers += [ptm.nn.Linear(1,1)]

        # Create model and set data:
        model = ptm.pytorch_model(
            layers,    
            save_options = {'initial_path': initial_path, 'name_notebook': name_notebook},
        )
        model.set_data(
            data_X=input_data, 
            data_Y=target_data, 
            data_X_validation=input_validation,
            data_Y_validation=target_validation
        )
        model.keep_track_params=False


        # -- TRAINING -- #
        import torch.optim as optim

        # train the model
        model.train(
            optimizer= optim.Adam,
            optimizer_options={'lr': 0.02},
            num_epochs = 100,                
            batch_size = 32,
            stop_training_options = {
                'lookback_epochs': 5, 
                'threshold_slope': 0.001,
                'threshold_std_dev': 0.2
            },
        )

        # -- SAVE -- #

        # dump the object
        model.dump()

        # save string of the model
        model.save_str(
            metadata={
                "data_file_path": data_file_path,
                "n_aminoacids": n_aminoacids,
                "block_n_layers": block_n_layers,
            }
        )

        # plot the losses of the trainig loop and save the figures
        model.plot_losses(save=True, save_txt=True)

        #save all the parameters of the model and the intermediate parameters of the model
        model.save_state_dict(intermediate=False)

        # plot the validation and save it
        model.plot_validation(save=True, fig_size=(9,9), percentatge=1)

        # print validation and save it
        model.str_validation(save=True, precision=4, percentatge=1, printing=False)

## --- Num:  4  --- ##


0  # ------- block_n_layers:  1  ------- #


76.0 1.0


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
# push changes to git
if initial_path != '../':
    !cd peptide-QML && git add . && git commit -m "data trained model" && git push